<center>
<img src="https://raw.githubusercontent.com/yoonkt200/python-data-analysis/master/img/this_is_data_anal.png" width="200" height="200"><br>
</center>

## 4.2 강남역 맛집 리뷰로 알아보는 감성 분류

분류 모델의 가장 대표적인 활용 방법 중 하나는 바로 감성 분류이다. 감성 분류란 문서를 긍정의견 또는 부정의견으로 나누어 분류하는 것이다. 이번엔 강남역 맛집 리뷰 데이터를 활용하여 감성 분류를 수행해보자.

### step.1 크롤링: 네이버 플레이스 리뷰 크롤링

첫 번째 단계에서는 크롤링을 이용하여 감성 분류에 필요한 데이터를 직접 수집하자. 이번에 크롤링할 데이터는 네이버 플레이스의 음식점 랭킹 정보이다. 네이버 플레이스 API는 별도의 라이브러리 설치나 개발자 등록 없이 source_url에 검색 규칙을 추가하는 것만으로도 크롤링을 적용할 수 있다. 아래의 코드와 같이 start, display, query, sortingOrder 파라미터로 검색 규칙을 추가하자.

- start, display : 검색 결과를 얼마만큼 보여줄지에 관련된 파라미터
- query : 검색하고 싶은 장소나 음식점에 대한 검색어
- sortingOrder : 어떤 방식으로 검색 결과를 정렬할지에 대한 파라미터

검색 규칙을 추가한 url_concat에 request.get 함수를 적용하면 json 형태의 검색 결과 데이터를 얻을 수 있다. 그리고 이를 jason.loads 함수로 변환하면 파이썬의 dictionary와 동일한 형태로 사용할 수 있다.

In [1]:
# -*- coding: utf-8-*-
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

### step.2 텍스트 전처리: 분류 모델 피처로 변환하기

이번 단계에는 텍스트 데이터를 감성 분류의 피처로 사용할 수 있도록 하는 텍스트 전처리 과정이다.

- 한글 텍스트로 전처리하기

In [2]:
import re

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [3]:
df = pd.read_csv("/home/jaeyoon89/python-data-analysis/chapter4/review_data.csv")
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x))
del df['review']
df.head()

,score,y,ko_text
0,5,1,친절하시고 깔끔하고 좋았습니다
1,5,1,조용하고 고기도 굿
2,4,1,갈비탕과 냉면 육회비빔밥이 맛있습니다
3,4,1,대체적으로 만족하나와인의 구성이 살짝 아쉬움
4,5,1,고기도 맛있고 서비스는 더 최고입니다


형태소를 추출하는 전처리 과정도 진행하자. 아래 코드에서 형태소/품사의 형태로 데이터를 추출하는 get_pos()라는 함수를 정의했고 이 함수에 대한 테스트 코드 실행 결과는 아래와 같다.

- 형태소 단위의 추출 함수 정의하기

In [4]:
from konlpy.tag import Okt

# konlpy라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

# 형태소 추출 동작을 테스트합니다.
result = get_pos(df['ko_text'][0])
print(result)

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# 형태소를 벡터 형태의 학습 데이터셋(X 데이터)으로 변환합니다.
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist()

SyntaxError: unexpected EOF while parsing (<ipython-input-5-f8aa91c73d72>, line 5)